# Data Science Short Course


### Session 1: Data Wrangling
Data wrangling is the process of cleaning, structuring and enriching raw data into a desired format for better decision making.

Credits:
* Mohammad Raza https://towardsdatascience.com/data-wrangling-with-pandas-5b0be151df4e
* Data source: [UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))
* Pandas functionality: [Documentation](https://pandas.pydata.org/)



---

### Step 1: Import
Lets import [pandas](https://pandas.pydata.org/) library.

In [ ]:
import pandas as pd

Load the dataset from the git. Alternatively you can upload you own data or use data from google drive. Check [here](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92) for more details.

Check the details of this data.

In [ ]:
url = 'https://raw.githubusercontent.com/moeraza/pandas_data_wrangling/master/data/breast_cancer_data.csv'
df = pd.read_csv(url)


Create a data frame.

In [ ]:
#Inspect the datatypes
df.dtypes

patient_id                 int64
clump_thickness          float64
cell_size_uniformity     float64
cell_shape_uniformity      int64
marginal_adhesion          int64
single_ep_cell_size        int64
bare_nuclei               object
bland_chromatin          float64
normal_nucleoli          float64
mitoses                    int64
class                     object
doctor_name               object
dtype: object

### Explanation of Output.
In the data we have the following columns as described by the source — Patient ID: id number, Clump Thickness: 1–10, Uniformity of Cell Size: 1–10, Uniformity of Cell Shape: 1–10, Marginal Adhesion: 1–10, Single Epithelial Cell Size: 1–10, Bare Nuclei: 1–10, Bland Chromatin: 1–10, Normal Nucleoli: 1–10, Mitoses: 1–10, Class: malignant or benign, Doctor name: 4 different doctors.

Based on this, we can assume that patient_id is a unique identifier, class is going to tell us whether the tumor is malignant (cancerous) or benign (not cancerous). The remaining columns are numeric medical descriptions of the tumor, except for the doctor_name which is a categorical feature.

Things to keep in mind — If our goal is to predict wether a tumor is cancerous or not based on the remaining features, we will have to one hot encode the categorical data and clean up the numerical data.

In [ ]:
#Check the top 5 rows and headers
df.head()

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,doctor_name
0,1000025,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Doe
1,1002945,5.0,4.0,4,5,7,10,3.0,2.0,1,benign,Dr. Smith
2,1015425,3.0,1.0,1,1,2,2,3.0,1.0,1,benign,Dr. Lee
3,1016277,6.0,8.0,8,1,3,4,3.0,7.0,1,benign,Dr. Smith
4,1017023,4.0,1.0,1,3,2,1,3.0,1.0,1,benign,Dr. Wong


In [ ]:
#Describe function of pandas to understand the spread of data values in each columns
df.describe()

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bland_chromatin,normal_nucleoli,mitoses
count,6.990000e+02,698.000000,698.000000,699.000000,699.000000,699.000000,695.000000,698.000000,699.000000
mean,1.071704e+06,4.416905,3.137536,3.207439,2.793991,3.216023,3.447482,2.868195,1.589413
std,6.170957e+05,2.817673,3.052575,2.971913,2.843163,2.214300,2.441191,3.055647,1.715078
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,3.500000,4.000000,5.000000,4.000000,1.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


This neatly summarizes some statistical data for all numerical columns.
 For categorical data we can hand this by grouping together values:

In [ ]:
df.groupby(by =['class', 'doctor_name']).size()

class      doctor_name
benign     Dr. Doe        127
           Dr. Lee        121
           Dr. Smith      102
           Dr. Wong       108
malignant  Dr. Doe         58
           Dr. Lee         60
           Dr. Smith       74
           Dr. Wong        49
dtype: int64

Some more functions to try and explore the data and make sense of it.

In [ ]:
#Its good to inspect your unique key identifier
df.nunique()

In [ ]:
# Here we list all columns
df.columns



---


## Dealing with missing values
With every dataset it is vital to evaluate the missing values.


*  How many are there? Is it an error?
*  Are there too many missing values?  
*  Does a missing value have a meaning relative to its context?

We can sum up the total missing values using the following:

In [ ]:
df.isna().sum()

patient_id               0
clump_thickness          1
cell_size_uniformity     1
cell_shape_uniformity    0
marginal_adhesion        0
single_ep_cell_size      0
bare_nuclei              2
bland_chromatin          4
normal_nucleoli          1
mitoses                  0
class                    0
doctor_name              0
dtype: int64

Now that we have identified our missing values, we have a few options.


### 1.   Fill them in with a certain value

> 1.   Zero, mean/max/median by column, string)
2.   Mean
3.   Max
4.   Median by column
5.   String

### 2.   Remove them by row.


  Since there are few missing values, we can drop the rows to avoid skewing the data in further analysis.

In [ ]:
df = df.dropna(axis = 0, how = 'any')

In [ ]:
#Rename columns
df.rename(index =str, columns = {'patient_id':'pid'})

,pid,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,doctor_name
0,1000025,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Doe
1,1002945,5.0,4.0,4,5,7,10,3.0,2.0,1,benign,Dr. Smith
2,1015425,3.0,1.0,1,1,2,2,3.0,1.0,1,benign,Dr. Lee
3,1016277,6.0,8.0,8,1,3,4,3.0,7.0,1,benign,Dr. Smith
4,1017023,4.0,1.0,1,3,2,1,3.0,1.0,1,benign,Dr. Wong
...,...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3.0,1.0,1,1,3,2,1.0,1.0,1,benign,Dr. Lee
695,841769,2.0,1.0,1,1,2,1,1.0,1.0,1,benign,Dr. Smith
696,888820,5.0,10.0,10,3,7,3,8.0,10.0,2,malignant,Dr. Lee
697,897471,4.0,8.0,6,4,3,4,10.0,6.0,1,malignant,Dr. Lee




---
### Inspecting duplicates


In [ ]:
# Its good to inspect unique key identifiers
df.nunique()

patient_id               637
clump_thickness           10
cell_size_uniformity      10
cell_shape_uniformity     10
marginal_adhesion         10
single_ep_cell_size       10
bare_nuclei               11
bland_chromatin           10
normal_nucleoli           10
mitoses                    9
class                      2
doctor_name                4
dtype: int64

We see here that although there are 699 rows, there are only 645 unique patient_id’s. This could mean that some patient appear more than once in the dataset. To isolate these patients and view their data, we use the following:

In [ ]:
df[df.duplicated(subset = 'patient_id', keep =False)].sort_values('patient_id')

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,doctor_name
267,320675,3.0,3.0,5,2,3,10,7.0,1.0,1,malignant,Dr. Wong
272,320675,3.0,3.0,5,2,3,10,7.0,1.0,1,malignant,Dr. Smith
575,385103,5.0,1.0,2,1,2,1,3.0,1.0,1,benign,Dr. Smith
269,385103,1.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Doe
271,411453,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Wong
...,...,...,...,...,...,...,...,...,...,...,...,...
560,1321942,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Doe
660,1339781,1.0,1.0,1,1,2,1,2.0,1.0,1,benign,Dr. Lee
661,1339781,4.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Smith
672,1354840,2.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Wong


This line displays all the duplicated patient_id’s in order. The number of times a patient shows up in the dataset can also be viewed.

In [ ]:
repeat_patients = df.groupby(by = 'patient_id').size().sort_values(ascending =False)

In [ ]:
repeat_patients

patient_id
1182404    6
1276091    5
1198641    3
1238777    2
385103     2
          ..
1212232    1
1211594    1
1211265    1
1211202    1
61634      1
Length: 645, dtype: int64



---
### If we want to remove patients that show up more that 2 times in the data set.


In [ ]:
filtered_patients = repeat_patients[repeat_patients > 1].to_frame().reset_index()
filtered_df = df[~df.patient_id.isin(filtered_patients.patient_id)]

In [ ]:
filtered_df.describe()

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bland_chromatin,normal_nucleoli,mitoses
count,5.990000e+02,598.000000,598.000000,599.000000,599.000000,599.000000,595.000000,598.000000,599.000000
mean,1.079525e+06,4.593645,3.272575,3.353923,2.956594,3.323873,3.563025,3.061873,1.622705
std,6.561027e+05,2.864399,3.079236,3.007507,2.959840,2.244715,2.472879,3.187788,1.757322
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.173514e+06,4.000000,1.000000,2.000000,1.000000,2.000000,3.000000,1.000000,1.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.750000,1.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [ ]:
filtered_df.nunique()

patient_id               599
clump_thickness           10
cell_size_uniformity      10
cell_shape_uniformity     10
marginal_adhesion         10
single_ep_cell_size       10
bare_nuclei               11
bland_chromatin           10
normal_nucleoli           10
mitoses                    9
class                      2
doctor_name                4
dtype: int64



---

### Reshaping Data
The dataset has elements of categorical data in the “doctor_name” column. To feed this data into a machine learning pipeline, we will need to convert it into a one hot encoded column. This can be done with a sci-kit learn package, however we will do it in pandas to demonstrate the pivoting and merging functionality. Start off by creating a new dataframe with the categorical data.

In [ ]:
categorical_df = df[['patient_id','doctor_name']]
categorical_df['doctor_count'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


We add a column an extra column to identify which doctor a patient deals with. Pivot this table so that we only have numerical values in the cells and the columns become the doctors’ name. Then fill in the empty cells with 0. This is known as one hot encoding.
![alt text](https://www.mwbrady.com/post/encodingcategoricaldata/Untitled-5f6d339d-ce4a-4533-8b67-c4e688b85ce0.png)

In [ ]:
doctors_one_hot_encoded = pd.pivot_table( categorical_df,
                                  index = categorical_df.index,
                                  columns = ['doctor_name'],
                                   values = ['doctor_count'] )
doctors_one_hot_encoded = doctors_one_hot_encoded.fillna(0)

In [ ]:
#drop multi index column
doctors_one_hot_encoded.columns = doctors_one_hot_encoded.columns.droplevel()

In [ ]:
doctors_one_hot_encoded.head()

doctor_name,Dr. Doe,Dr. Lee,Dr. Smith,Dr. Wong
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0


We can now join this back to our main table. Typically a left join in pandas looks like this:

```
leftJoin_df = pd.merge(df1, df2, on ='col_name', how='left')
```
However we are joining on the index so we pass the “left_index” and “right_index” option to specify that the join key is the index of both tables

In [ ]:
combined_df = pd.merge(df, doctors_one_hot_encoded, left_index = True,right_index =True, how ='left')

In [ ]:
#drop the column that we no longer need
combined_df = combined_df.drop(columns=['doctor_name'])

In [ ]:
combined_df.head()

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,Dr. Doe,Dr. Lee,Dr. Smith,Dr. Wong
0,1000025,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,1.0,0.0,0.0,0.0
1,1002945,5.0,4.0,4,5,7,10,3.0,2.0,1,benign,0.0,0.0,1.0,0.0
2,1015425,3.0,1.0,1,1,2,2,3.0,1.0,1,benign,0.0,1.0,0.0,0.0
3,1016277,6.0,8.0,8,1,3,4,3.0,7.0,1,benign,0.0,0.0,1.0,0.0
4,1017023,4.0,1.0,1,3,2,1,3.0,1.0,1,benign,0.0,0.0,0.0,1.0




---

### Row-wise Operations

Another key component in data wrangling is having the ability to conduct row-wise or column wise operations. Examples of this are; rename elements within a column based on its value and create a new column that yields a specific value based on multiple attributes within the row.

For this example lets create a new column that categorizes a patients cell as normal or abnormal based on its attributes. We first define our function and the operation that it will be doing.

In [ ]:
# Feature building:

def celltypelabel(x):
    if ((x['cell_size_uniformity'] > 5) & (x['cell_shape_uniformity'] > 5)):
        return('normal')
    else:
        return('abnormal')

Then we use the pandas apply function to run the celltypelabel(x) function on the dataframe.

In [ ]:
combined_df['cell_type_label'] = combined_df.apply(lambda x: celltypelabel(x), axis=1)

In [ ]:
combined_df.head()

,patient_id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,Dr. Doe,Dr. Lee,Dr. Smith,Dr. Wong,cell_type_label
0,1000025,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,1.0,0.0,0.0,0.0,abnormal
1,1002945,5.0,4.0,4,5,7,10,3.0,2.0,1,benign,0.0,0.0,1.0,0.0,abnormal
2,1015425,3.0,1.0,1,1,2,2,3.0,1.0,1,benign,0.0,1.0,0.0,0.0,abnormal
3,1016277,6.0,8.0,8,1,3,4,3.0,7.0,1,benign,0.0,0.0,1.0,0.0,normal
4,1017023,4.0,1.0,1,3,2,1,3.0,1.0,1,benign,0.0,0.0,0.0,1.0,abnormal


## Conclusion
Although some of these data manipulation steps can be done in SAS and excel. Doing it in python not only allows you to connect the data to vast open source resources in computer vision, machine and deep learning, but also for ETL automation purposes and more.